In [1]:
datafile = '/home/wbm001/deeplpi/DeepLPI/data/seq_dict.csv'
datapath = "/home/wbm001/deeplpi/DeepLPI/data/"
modelpath = '/home/wbm001/deeplpi/DeepLPI/Preprocess/prose/prose_mt_3x1024.sav'

In [2]:
import pandas as pd
data = pd.read_csv(datafile)
data = data[["0","ki_test","kd_test"]]
data

,0,ki_test,kd_test
0,MVAHDEIGGLLPIKRTIRVLDVNNQPFREQEEPSNKRVRPLGRVTS...,False,False
1,MLPPGRNRTAQPARLGLQRQLAQVDAPAGSATPLGPAQVVTAGLLT...,False,False
2,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,False,False
3,MKSILDGLADTTFRTITTDLLYVGSNDIQYEDIKGDMASKLGYFPQ...,False,False
4,MSARLNNLLQHIAVKDKDSDTMRHLKQRMALASLANQFTVGKDHLK...,False,False
...,...,...,...
4353,MSEPRFVHLRVHSDYSMIDGLAKTAPLVKKAAALGMPALAITDFTN...,False,False
4354,MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYM...,False,False
4355,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,False,False
4356,MITTQMWHFYVTRVGLLLLISILPGTTGQGESRRQEPGDFVKQDIG...,False,False


In [3]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from prose.models.multitask import ProSEMT # use the path of ProSE
from prose.alphabets import Uniprot21

model = ProSEMT.load_pretrained(modelpath)
model.eval()
model = model.cuda()

def embed_sequence(x):
    n = model.embedding.proj.weight.size(1)

    alphabet = Uniprot21()
    x = x.upper()
    x = alphabet.encode(x)
    x = torch.from_numpy(x)

    x = x.cuda()

    with torch.no_grad():
        x = x.long().unsqueeze(0)
        z = model.transform(x)
        z = z.sum(0)
        z = z.cpu().numpy()

    return z

In [4]:
import numpy as np

print("start to embed")

embed = np.array(list(data["0"].map(lambda x : embed_sequence(x.encode())))).reshape(-1,1)

print("embed down, start to write file")

start to embed


: 

: 

In [ ]:
embed.shape

NameError: name 'embed' is not defined

In [7]:
import csv
 
with open(datapath + 'seq_embed.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(embed.tolist())

In [8]:
embed

array([[ 31.        ,  45.        ,  19.        , ...,   0.49488112,
        116.883255  ,  58.524834  ],
       [ 41.        ,  26.        ,  17.        , ..., -13.49969   ,
         89.910965  ,  71.94673   ],
       [ 81.        ,  41.        ,  20.        , ...,  68.391655  ,
        244.59235   ,  32.140823  ],
       ...,
       [ 43.        ,  42.        ,  21.        , ..., -31.050955  ,
         24.48243   ,  -8.320545  ],
       [ 33.        ,  21.        ,  18.        , ..., -11.8458395 ,
         77.132645  , 221.38464   ],
       [ 72.        ,  61.        ,  44.        , ...,  -9.280804  ,
        234.239     , 266.6541    ]], dtype=float32)